# 理論式を用いたインダクタンスの算出

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import numpy as np
from tqdm import tqdm_notebook as tqdm
import os
import scipy                                                                                                                           
import math
import scipy.optimize as opt
import sympy as sy

矩形コイルのインダクタンスの理論式
$$
L(nH) = 1.62\times10^{-3}\times{d_{out}}^{-1.21}w^{-0.147}d_{avg}^{2.40}n^{1.78}s^{-0.030}
$$
に従って算出する。
なお、各変数の単位はL(nH), Dout, w, Davg, sはそれぞれumである。

所望のインダクタンスLobg

In [84]:
Lobj = 287*10**(3)
Lobj

287000

In [85]:
# 各種係数
# インダクタンス理論式
Dout = 30000 # 外径(単位um)
B = 1.62*(10**(-3))
a_dout = -1.21
a_w = -0.147
a_davg = 2.40
a_n = 1.78
a_s = -0.030

# 抵抗率理論式
sigma = 1.74*10**(-8) # 銅の抵抗率
t = 35 # コイル銅箔厚さ(um)

In [86]:
# # 単なる式の確認
# n=125

# Dout=30*(10**(3))
# w=0.06*(10**(3))
# s=0.06*(10**(3))
# Din=Dout-w*(2*n)-s*(2*n-1) # ここは図的にこうなる
# Davg=(Dout+Din)/2

ラグランジュの未定乗数法を用いる
目的関数(最小値を求めたい、ここではコイルの抵抗L_R)は以下のFの式で表される(t, σは定数, lengthはn, w, sの関数)
$$
F(n, w, s) = \frac{\sigma}{tw}\times length(n, w, s)
$$
拘束条件(所望のインダクタンス)はLobgを所望のインダクタンスとおくと、
$$
G(n, w, s) = 1.62\times10^{-3}\times{d_{out}}^{-1.21}w^{-0.147}d_{avg}^{2.40}n^{1.78}s^{-0.030}-L_{obj}=0
$$
未定乗数hを用いて
$$
\frac{\partial}{\partial n}(F+h G)=0
$$
$$
\frac{\partial}{\partial w}(F+h G)=0
$$
$$
\frac{\partial}{\partial s}(F+h G)=0
$$
以上4つの式で解(n, w, s, h)を求める。

変数を定義

In [87]:
w, s, n, Din, Dav, h = sy.symbols("w s n Din Dav h")

目的関数F(抵抗値の理論式)を定義

(注: まだlengthは正確な定義まで練ってない, ひとまずこれくらいの値)
$$
length= 4(D_{out} -2n(s+w)-w)
$$

In [98]:
length = (Dout - 2*n*(s+w)-w)*4 # コイルの総長さ
F= (sigma*length*10**(-6))/((w*10**(-6))*(t*10**(-6)))

28571428571.4286*(-1.392e-13*n*(s + w) - 6.96e-14*w + 2.088e-9)/w

制約条件Gを定義

In [89]:
Din=Dout-w*(2*n)-s*(2*n-1)
Dav=(Dout+Din)/2
G = B*(Dout**a_dout)*(w**a_w)*(Dav**a_davg)*(n**a_n)*(s**a_s)-Lobj
L = B*(Dout**a_dout)*(w**a_w)*(Dav**a_davg)*(n**a_n)*(s**a_s)

w, sに応じて、巻き数の上限は決定する

In [92]:
nmax = (Dout+s)/(2*(s+w))

ラグランジュの定理式に代入
ここではhを未定乗数として扱っている

In [29]:
# theor = F - h * G

In [30]:
# # 各変数で偏微分する
# diffn = sy.diff(theor, n)
# diffw = sy.diff(theor, w)
# diffs = sy.diff(theor, s)
# diffh = sy.diff(theor, h)
# type(G)

In [31]:
# 連立方程式を解く
# res = sy.solve([diffn, diffw, diffs, diffh])

In [32]:
# def F(x):
#     return [
#         complex(diffn.subs([(n, x[0]), (w, x[1]),(s, x[2]), (h, x[3])])),
#         complex(diffw.subs([(n, x[0]), (w, x[1]),(s, x[2]), (h, x[3])])),
#         complex(diffs.subs([(n, x[0]), (w, x[1]),(s, x[2]), (h, x[3])])),
#         complex(diffh.subs([(n, x[0]), (w, x[1]),(s, x[2]), (h, x[3])]))
#          ]

# guess = [0,0,0,0]

# sol = opt.newton_krylov(F, guess)

# print(sol)

### ラグランジュでやらず、pythonのoptimize.minimize関数を用いた

In [101]:
# 代入用関数
def Gx(x):
    return G.subs([(n, x[0]), (w, x[1]),(s, x[2])])

def Fx(x):
    return F.subs([(n, x[0]), (w, x[1]),(s, x[2])])

def nmaxx(x):
    return nmax.subs([(w, x[1]),(s, x[2])])

def Lx(x):
    return L.subs([(n, x[0]), (w, x[1]),(s, x[2])])

def lengthx(x):
    return length.subs([(n, x[0]), (w, x[1]),(s, x[2])])

# 制約条件
cons = (
        {"type":"eq","fun": lambda x: Gx(x) - 0}, # 所望のインダクタンスに一致
        {"type": "ineq", "fun": lambda x: nmaxx(x) - x[0]}, # 巻き数上限以下のn
        {"type": "ineq", "fun": lambda x: lengthx(x) - 0} # コイル長は0以上
       )

# 結果格納用のdf
columns = ["i","n", "w","s"]
res_df = pd.DataFrame(columns=columns)

# 適当に全パラメータ1~100で動かしてみた(妥当性不明)
for i in range(1, 100):
    x0 = [i, i, i]
    try:
        result = opt.minimize(fun = opt_F,
                          x0=x0, 
                          bounds =  ((1,None), (10**(-8),None), (10**(-8),None)),
                          constraints= cons,
                          method="SLSQP")
        res = list(result.x)
        res_df = res_df.append({'i': i,'n': res[0], 'w': res[1], 's': res[2],"L": Lx(res), "R_L": Fx(res)}, ignore_index=True)
    except TypeError:
        continue

In [102]:
res_df

,i,n,w,s,L,R_L
0,1,133.808668,111.683030,0.000007,287000.000000024,-1.82914356209622e-16
1,2,144.132749,103.710172,0.000782,286999.999999686,7.37548009591051e-16
2,3,151.887813,98.410783,0.022358,286999.999999997,8.93653640671177e-17
3,4,161.507065,42.765209,49.977591,287000.000000003,4.44089209850063e-16
4,5,165.376679,65.569991,24.933791,287000.000000001,9.43689570931383e-15
...,...,...,...,...,...,...
94,95,89.208112,98.093261,0.000398,287600.244954652,0.251384181133196
95,96,89.315777,98.165670,0.000400,287578.913249772,0.250507379904312
96,97,88.889396,96.679609,0.000626,287541.827946382,0.261543553894560
97,98,87.981806,95.462204,0.000755,287462.523594086,0.273021903409079


In [103]:
res_df['R_L'].min()

-1.07045918997125e-14

内部抵抗の最小値が負なのは何で...